<h1>Lab Three: Extending Logistic Regression</h1>
<b>By Michael Watts, Maya Muralidhar, Nora Potenti, and Adam Ashcraft </b>

# 1.0 Business Understanding

This dataset is from the Free Music Archive, a collection of legally available audio files.  The numerical data is made up of features extracted from a musical analysis python package called librosa that quantifies some of the characteristics of an mp3 and also includes statistics such as mean, skew, and kurtosis.  The dataset also includes unique codes for genres of music.  We will use these features to determine what the genre of a piece is given the librosa feature extraction.  This classification would prove useful for a music streaming application such as Spotify that would want to integrate new music into its platform quickly, especially if the defined genre in the audio file's metadata doesn't matches one of the genres defined in the application's database. It would also help with the application's recommendation system; by broadly defining the main genre categories, users could receive recommendations that are audibly similar. For this use case, the model would be deployed to a production. To measure our success, we will compare our results to the International Society for Music Information Retrieval’s Music Information Retrieval Evaluation eXchange (MIREX) 2017 competition winning algorithm LPNKK1, which has an accuracy of approximately 77%. Our algorithm is considered a success if it can outperform this academic algorithm’s genre classification success rate.
<hr>
Data Set Source: https://github.com/mdeff/fma <br>
MIREX Competition Results: http://www.music-ir.org/nema_out/mirex2017/results/act/mixed_report/summary.html

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import pickle
import warnings
from sklearn.metrics import accuracy_score

In [ ]:
feature_df = pd.read_csv("data/features.csv", skiprows=range(1,4))
feature_df.rename(columns={'feature':'track_id'}, inplace=True)
genre_df = pd.read_csv("data/genres.csv")
track_df = pd.read_csv("data/tracks.csv", skiprows=[0,2])
track_df.rename(columns={'Unnamed: 0':'track_id'}, inplace=True)
feature_df.head()

In [ ]:
genre_df.head()

In [ ]:
track_df.columns

In [ ]:
track_df[['genre_top', 'genres']].head()

In [ ]:
top_ten_genres = genre_df.sort_values(by='#tracks', ascending=False)[:10]
top_ten_genres

We'll only look at the top ten genres listed and, if the track's top genre is in this list, include that track in our reduced dataframe.

In [ ]:
track_df = track_df[track_df['genre_top'].isin(top_ten_genres['title'].values)]
track_df['genre_top'].value_counts()

In [ ]:
feature_df = feature_df.set_index('track_id').join(track_df[['track_id', 'genre_top']].set_index('track_id'))
feature_df.dropna(how='any', axis=0, inplace=True)
feature_df.head()

With our label determined, let's prepare the data for our model.

In [ ]:
label_mapping = dict(zip(feature_df['genre_top'].unique(), range(0,10)))
y = feature_df['genre_top'].map(label_mapping, na_action='ignore').values
X = feature_df.drop(columns=['genre_top'])

# 2.0 Evaluation

# 2.1 Accuracy
For our accuracy score, we are using F-Measure. This accuracy score is controlled by the actual positive values, the false positives, and the false negatives.  A high F-Measure represents low false positives and negatives. This accuracy score best fits our given data set. False positives and false negatives are equally problematic in our data set. If we mischaracterize too many songs’ genres as Genre A; suddenly, Genre A gets flooded with hundreds of songs Genre A fans do not want to listen to. The service is no longer able to provide users with music that is audibly similar. If Band A produces a lot of songs that are Genre A but classified as Genre B, then users who listen to Genre A are unable to find songs by Band A, which is bad for the artist. Users who listen to Genre B are flooded with mischaracterized songs they do not want to listen to. Again, the service is no longer able to provide users with music that is audibly similar. All of these cases make a music streaming service into an unreliable source of new music and would drive users away. Since both of these cases are equally bad, we will use an accuracy score that weighs itself by both cases equally.   


In [ ]:
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
my_scorer = make_scorer(f1_score, average = 'micro')

# 2.2 Cross-Validation
Before we determine how we are going to cross-validate the classes, we should get some more information of the class division of our data.


In [ ]:
def numInY(Y):
    count_0 = 0
    count_1 = 0
    count_2 = 0
    count_3 = 0
    count_4 = 0
    count_5 = 0
    count_6 = 0
    for i in y:
        if i == 0:
            count_0 = count_0 + 1
        if i == 1:
            count_1 = count_1 + 1
        if i == 2:
            count_2 = count_2 + 1
        if i == 3:
            count_3 = count_3 + 1
        if i == 4:
            count_4 = count_4 + 1
        if i == 5:
            count_5 = count_5 + 1
        if i == 6:
            count_6 = count_6 + 1
    return [count_0, count_1, count_2, count_3, count_4, count_5, count_6] 
            
plt.pie(x=numInY(y),autopct='%1.1f%%')

As we can see, there is a massive class imbalance, where two classes alone make up over 50% of the data itself. In order to maintain even class division, any form of cross validation we choose will have to stratify the data. Otherwise we may get entire sets of data that only contain one class. This could skew our model’s training to just recognize everything as a single class, as in a fold of the data with only one class, this would be the most correct assumption. We also have another issue; since, the individual songs are grouped together by artist, we could get folds that have a dispositional amount of songs by the same artist in one genre. Band A and Band B may both be Genre C but have different sounding songs. We do not want a model only trained to recognize Band A as Genre C. In order to combat this, we must also shuffle the data in our cross validation. For these reasons, we are using a stratified shuffle split. This is an accurate reflection of real world test cases, where both a variety of songs from different artist and different genres will be tested against the model. Music streaming services are constantly getting new songs from new artist from a wide variety of different genres. However, some genres are more popular than others and therefore will have more songs belonging to them. This is bias towards genres is reflected in the stratification of the model’s folds. Musicians across the globe are able to produce and submit songs to music streaming services. These musicians while working in the same genre will not have same influences and could produce songs of the same genre that do not exactly like one another. To reflect the difference is artist, we shuffle the data, ensuring no fold is too focused on one specific artist’s definition of a genre. 
     


In [ ]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
cv = StratifiedShuffleSplit(n_splits=10, random_state=1) 

In [ ]:
class MultiLayerPerceptronBase(object):
    def __init__(self, layers=2, phi='linear', cost_fn='quadratic', n_hidden=30,
                 C=0.0, epochs=500, eta=0.001, random_state=None):
        np.random.seed(random_state)
        self.layers=layers
        self.phi = phi
        self.cost_fn = cost_fn
        self.n_hidden = n_hidden
        self.l2_C = C
        self.epochs = epochs
        self.eta = eta

    @staticmethod
    def _encode_labels(y):
        """Encode labels into one-hot representation"""
        onehot = pd.get_dummies(y).values.T

        return onehot

    def _initialize_weights(self):
        """Initialize weights with small random numbers."""
        weights = []
        W1_num_elems = (self.n_features_ + 1) * self.n_hidden
        W1 = np.random.uniform(-1.0, 1.0, size=W1_num_elems)
        W1 = W1.reshape(self.n_hidden, self.n_features_ + 1)  # reshape to be W
        weights.append(W1)
        for i in range(1, self.layers-1):
            W_num_elems = ((weights[i-1].shape[0]+1)*(self.n_hidden))
            W = np.random.uniform(-1.0, 1.0, size=W_num_elems)
            W = W.reshape(self.n_hidden, weights[i-1].shape[0]+1)
            weights.append(W)
        Wfinal_num_elems = (self.n_hidden + 1) * self.n_output_
        Wfinal = np.random.uniform(-1.0, 1.0, size=Wfinal_num_elems)
        Wfinal = Wfinal.reshape(self.n_output_, self.n_hidden + 1)  # reshape to be W
        weights.append(Wfinal)
        return weights

    @staticmethod
    def _sigmoid(phi, z):
        """Use scipy.special.expit to avoid overflow"""
        # 1.0 / (1.0 + np.exp(-z))
        if phi == 'sigmoid':
            return expit(z)
        elif phi == 'relu':
            return np.maximum(z, 0, z)
        elif phi == 'silu':
            return z * expit(z)
        else:
            return z

    @staticmethod
    def _add_bias_unit(X, how='column'):
        """Add bias unit (column or row of 1s) to array at index 0"""
        if how == 'column':
            ones = np.ones((X.shape[0], 1))
            X_new = np.hstack((ones, X))
        elif how == 'row':
            ones = np.ones((1, X.shape[1]))
            X_new = np.vstack((ones, X))
        return X_new

    @staticmethod
    def _L2_reg(lambda_, weights):
        """Compute L2-regularization cost"""
        # only compute for non-bias terms
        return (lambda_ / 2.0) * np.sqrt(sum(np.mean(weight[:, 1:] ** 2) for weight in weights))

    def _cost(self, A, Y_enc, weights):
        '''Get the objective function value'''
        if self.cost_fn == 'cross_entropy':
            cost = -np.mean(np.nan_to_num((Y_enc * np.log(A) + (1 - Y_enc) * np.log(1 - A))))
        else:
            cost = np.mean((Y_enc - A) ** 2)
        L2_term = self._L2_reg(self.l2_C, weights)
        return cost + L2_term

    def _feedforward(self, X, weights):
        """Compute feedforward step
        """
        A = []
        Z = []
        A1 = self._add_bias_unit(X, how='column')
        A1 = A1.T
        Z1 = weights[0] @ A1
        A.append(A1)
        Z.append(Z1)
        for i in range(1, len(weights)):
            A_new = self._sigmoid(self.phi, Z1)
            A_new = self._add_bias_unit(A_new, how='row')
            Z_new = weights[i] @ A_new
            A.append(A_new)
            Z.append(Z_new)
        A_final = self._sigmoid(self.phi, Z[-1])
        A.append(A_final)
        return A, Z

    def _get_gradient(self, A, Z, Y_enc, weights):
        """ Compute gradient step using backpropagation.
        """
        # vectorized backpropagation
        gradients = []
        V = []

        if self.cost_fn == 'cross_entropy':
            Vfinal = (A[-1] - Y_enc)
        else:
            Vfinal = -2 * (Y_enc - A[-1]) * A[-1] * (1 - A[-1])  # last layer sensitivity
        V.append(Vfinal)

        gradfinal = Vfinal @ A[-2].T  # no bias on final layer
        #gradfinal = gradfinal.clip(-.25, .25)
        gradients.append(np.clip(gradfinal, -.25, .25))

        Vsecond_to_last = A[-2] * (1 - A[-2]) * (weights[-1].T @ V[0])  # back prop the sensitivity
        V.append(Vsecond_to_last)
        gradsecond_to_last = V[-1][1:, :] @ A[-3].T  # dont back prop sensitivity of bias

        gradients.append(np.clip(gradsecond_to_last, -.25, .25))
        for i in range(2, len(A)-1):
            Vcurr =  A[-i-1] * (1 - A[-i-1]) * (weights[-i].T @ V[i-1][1:, :])  # back prop the sensitivity
            V.append(Vcurr)
            gradcurr = Vcurr[1:, :] @ A[-i-2].T
            gradcurr = gradcurr.clip(-.25, .25)
            gradients.append(gradcurr)
        gradients.reverse()

        # regularize weights that are not bias terms
        gradients[0][:, 1:] += weights[0][:, 1:] * self.l2_C
        for weight_index in range(1, len(weights)-1):
            gradients[weight_index][:, 1:] += weights[weight_index][:, 1:] * self.l2_C
        gradients[-1][:, 1:] += weights[-1][:, 1:] * self.l2_C

        return gradients

    def predict(self, X):
        """Predict class labels"""
        A, Z = self._feedforward(X, self.weights)
        y_pred = np.argmax(A[-1], axis=0)
        return y_pred

# just start with the vectorized version and minibatch
class MLP(MultiLayerPerceptronBase):
    def __init__(self, alpha=0.0, decrease_const=0.0, shuffle=True,
                 minibatches=1, **kwds):
        # need to add to the original initializer
        self.alpha = alpha
        self.decrease_const = decrease_const
        self.shuffle = shuffle
        self.minibatches = minibatches
        # but keep other keywords
        super().__init__(**kwds)

    def fit(self, X, y, print_progress=False):
        """ Learn weights from training data. With mini-batch"""
        X_data, y_data = X.copy(), y.copy()
        Y_enc = self._encode_labels(y)

        # init weights and setup matrices
        self.n_features_ = X_data.shape[1]
        self.n_output_ = Y_enc.shape[0]
        self.weights = self._initialize_weights()

        delta_weights_prev =  [np.zeros(weight.shape) for weight in self.weights]

        self.cost_ = []
        self.score_ = []
        # get starting acc
        self.score_.append(f1_score(y_data, self.predict(X_data), average='micro'))
        for i in range(self.epochs):

            # adaptive learning rate
            self.eta /= (1 + self.decrease_const * i)

            if print_progress > 0 and (i + 1) % print_progress == 0:
                sys.stderr.write('\rEpoch: %d/%d' % (i + 1, self.epochs))
                sys.stderr.flush()

            if self.shuffle:
                idx_shuffle = np.random.permutation(y_data.shape[0])
                X_data, Y_enc, y_data = X_data[idx_shuffle], Y_enc[:, idx_shuffle], y_data[idx_shuffle]

            mini = np.array_split(range(y_data.shape[0]), self.minibatches)
            mini_cost = []
            for idx in mini:
                # feedforward
                A, Z = self._feedforward(X_data[idx],self.weights)

                cost = self._cost(A[-1], Y_enc[:, idx], self.weights)
                mini_cost.append(cost)  # this appends cost of mini-batch only

                # compute gradient via backpropagation
                gradients = self._get_gradient(A=A, Z=Z, Y_enc=Y_enc[:, idx],
                                                  weights=self.weights)

                # momentum calculations
                delta_weights = [self.eta * grad for grad in gradients]
                for weight_index in range(0, len(self.weights)):
                    self.weights[weight_index] -= (delta_weights[weight_index] + (self.alpha * delta_weights_prev[weight_index]))
                delta_weights_prev = delta_weights

            self.cost_.append(mini_cost)
            self.score_.append(f1_score(y_data, self.predict(X_data), average='micro'))

        return self